In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = 'C:/Users/User/Documents/Python Scripts/kaggle/ml100marathon-02-01/'
df_train = pd.read_csv(data_path + 'train_offline.csv')
df_test = pd.read_csv(data_path + 'test_offline.csv')

In [4]:
df_train.shape

(1160742, 7)

In [5]:
df_test.shape

(594142, 6)

In [4]:
df_test = df_test[~df_test.Date_received.isna()]
df_test.shape

(306313, 6)

In [7]:
df_train.head(20)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [8]:
df_test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,NaN,NaN,0.0,NaN
3,1439408,2632,8591.0,20:1,0.0,20160516.0
4,2029232,450,1532.0,30:5,0.0,20160530.0


In [5]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
na_check(df_train)

,Missing Ratio
Date,60.653703
Date_received,35.647284
Discount_rate,35.647284
Coupon_id,35.647284
Distance,6.015635


In [6]:
na_check(df_test)

,Missing Ratio
Distance,11.810468


In [7]:
def get_label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

df_train["label"] = df_train.apply(get_label, axis=1)

In [8]:
df_train["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [9]:
train_Y = df_train["label"]
train_num = train_Y.shape[0]

In [11]:
#D21
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

1160742

In [24]:
# NAIBE BAYES
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(np.asarray(x_train),np.asarray(y_train).ravel())
predicted= model.predict(np.asarray(x_vali))
print('Naive Bayes',accuracy_score(np.asarray(y_vali), predicted))

Naive Bayes 0.79


In [26]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(x_train,y_train.values.ravel())
predicted = knn_model.predict(x_vali)
print('KNN',accuracy_score(y_vali, predicted))

KNN 0.88


In [28]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(n_estimators = 100,random_state = 8)
rfc_model.fit(x_train,y_train.values.ravel())
predicted = rfc_model.predict(x_vali)
print('Random Forest',accuracy_score(y_vali, predicted))

Random Forest 0.87


In [29]:
#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver = 'saga')
lr_model.fit(x_train,y_train.values.ravel())
lr_predicted = lr_model.predict(x_vali)
print('Logistic Regression',accuracy_score(y_vali, lr_predicted))

Logistic Regression 0.83


In [30]:
#SVM
from sklearn.svm import SVC

svc_model = SVC(gamma = 'auto')
svc_model.fit(x_train,y_train.values.ravel())
svc_predicted = svc_model.predict(x_vali)
print('SVM',accuracy_score(y_vali, svc_predicted))

SVM 0.895


In [31]:
#DECISON TREE
from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier()
dtree_model.fit(x_train,y_train.values.ravel())
dtree_predicted = dtree_model.predict(x_vali)
print('Decision Tree',accuracy_score(y_vali, dtree_predicted))

Decision Tree 0.715


In [33]:
#XGBOOST
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(x_train,y_train.values.ravel())
xgb_predicted = xgb.predict(x_vali)
print('XGBoost',accuracy_score(y_vali, xgb_predicted))

XGBoost 0.855


In [53]:
rfc_model = RandomForestClassifier(n_estimators = 200,random_state = 8)
rfc_model.fit(df_train,df_label.values.ravel())
predicted = rfc_model.predict(df_test)
predicted.shape

(9000,)

In [73]:
pred = pd.DataFrame(predicted)
df_pred = pd.DataFrame({'Id': pred.index + 1, 'Solution': predicted})

In [77]:
df_pred.to_csv('rfc_1stTry.csv',index=False)

# After Feature Scaling

In [35]:
from sklearn.preprocessing import StandardScaler, Normalizer

norm = Normalizer()
#x_norm_train = norm.fit_transform(x_train)
#x_norm_test = norm.transform(x_vali)
norm_train_data = norm.fit_transform(df_train)

In [37]:
from sklearn.model_selection import cross_val_score
# NAIBE BAYES
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
#nb_model.fit(x_norm_train,y_train.values.ravel())
#nb_predicted= nb_model.predict(x_norm_test)
#print('Naive Bayes',accuracy_score(y_vali, nb_predicted))
print('Naive Bayes',cross_val_score(nb_model,norm_train_data, df_label.values.ravel(), cv=10).mean())

#KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors = 5)
#knn_model.fit(x_norm_train,y_train.values.ravel())
#knn_predicted= knn_model.predict(x_norm_test)
#print('KNN',accuracy_score(y_vali, knn_predicted))
print('KNN',cross_val_score(knn_model,norm_train_data, df_label.values.ravel(), cv=10).mean())

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(n_estimators = 100,random_state = 99)
#rfc_model.fit(x_norm_train,y_train.values.ravel())
#rfc_predicted = rfc_model.predict(x_norm_test)
#print('Random Forest',accuracy_score(y_vali,rfc_predicted))
print('Random Forest',cross_val_score(rfc_model,norm_train_data, df_label.values.ravel(), cv=10).mean())

#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver = 'saga')
#lr_model.fit(x_norm_train,y_train.values.ravel())
#lr_predicted = lr_model.predict(x_norm_test)
#print('Logistic Regression',accuracy_score(y_vali, lr_predicted))
print('Logistic Regression',cross_val_score(lr_model,norm_train_data, df_label.values.ravel(), cv=10).mean())

#SVM
from sklearn.svm import SVC

svc_model = SVC(gamma = 'auto')
#svc_model.fit(x_norm_train,y_train.values.ravel())
#svc_predicted = svc_model.predict(x_norm_test)
#print('SVM',accuracy_score(y_vali, svc_predicted))
print('SVM',cross_val_score(svc_model,norm_train_data, df_label.values.ravel(), cv=10).mean())

#DECISON TREE
from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier()
#dtree_model.fit(x_norm_train,y_train.values.ravel())
#dtree_predicted = dtree_model.predict(x_norm_test)
#print('Decision Tree',accuracy_score(y_vali, dtree_predicted))
print('Decision Tree',cross_val_score(dtree_model,norm_train_data, df_label.values.ravel(), cv=10).mean())

#XGBOOST
from xgboost import XGBClassifier

xgb = XGBClassifier()
#xgb.fit(x_norm_train,y_train.values.ravel())
#xgb_predicted = xgb.predict(x_norm_test)
#print('XGBoost',accuracy_score(y_vali, xgb_predicted))
print('XGBoost',cross_val_score(xgb,norm_train_data, df_label.values.ravel(), cv=10).mean())

Naive Bayes 0.808
KNN 0.9019999999999999
Random Forest 0.8699999999999999
Logistic Regression 0.8220000000000001
SVM 0.808
Decision Tree 0.7949999999999999
XGBoost 0.8710000000000001


# After Principal Component Analysis

In [39]:
from sklearn.decomposition import PCA

pca  = PCA(n_components=12)
#x_train = pca.fit_transform(x_train)
#x_test = pca.transform(x_test)
pca_train_data = pca.fit_transform(df_train)
explained_variance = pca.explained_variance_ratio_ 
print(explained_variance)

[0.25054403 0.2055048  0.08026473 0.05033658 0.04895951 0.04489904
 0.0417078  0.03127929 0.02309811 0.02100077 0.01619294 0.01266888]


In [40]:
pca_train_data.shape

(1000, 12)

In [41]:
# NAIBE BAYES
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
#nb_model.fit(pca_train_data,y_train.values.ravel())
#nb_predicted= nb_model.predict(x_norm_test)
#print('Naive Bayes',accuracy_score(y_test, nb_predicted))
print('Naive Bayes',cross_val_score(nb_model,pca_train_data, df_label.values.ravel(), cv=10).mean())

#KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors = 5)
#knn_model.fit(pca_train_data,y_train.values.ravel())
#knn_predicted= knn_model.predict(x_norm_test)
#print('KNN',accuracy_score(y_test, knn_predicted))
print('KNN',cross_val_score(knn_model,pca_train_data, df_label.values.ravel(), cv=10).mean())

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(n_estimators = 100,random_state = 99)
#rfc_model.fit(pca_train_data,y_train.values.ravel())
#rfc_predicted = rfc_model.predict(x_norm_test)
#print('Random Forest',accuracy_score(y_test,rfc_predicted))
print('Random Forest',cross_val_score(rfc_model,pca_train_data, df_label.values.ravel(), cv=10).mean())

#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver = 'saga')
#lr_model.fit(pca_train_data,y_train.values.ravel())
#lr_predicted = lr_model.predict(x_norm_test)
#print('Logistic Regression',accuracy_score(y_test, lr_predicted))
print('Logistic Regression',cross_val_score(lr_model,pca_train_data, df_label.values.ravel(), cv=10).mean())

#SVM
from sklearn.svm import SVC

svc_model = SVC(gamma = 'auto')
#svc_model.fit(x_norm_train,y_train.values.ravel())
#svc_predicted = svc_model.predict(x_norm_test)
#print('SVM',accuracy_score(y_test, svc_predicted))
print('SVM',cross_val_score(svc_model,pca_train_data, df_label.values.ravel(), cv=10).mean())

#DECISON TREE

from sklearn.tree import DecisionTreeClassifier

dtree_model = DecisionTreeClassifier()
#dtree_model.fit(x_norm_train,y_train.values.ravel())
#dtree_predicted = dtree_model.predict(x_norm_test)
#print('Decision Tree',accuracy_score(y_test, dtree_predicted))
print('Decision Tree',cross_val_score(dtree_model,pca_train_data, df_label.values.ravel(), cv=10).mean())

#XGBOOST
from xgboost import XGBClassifier

xgb = XGBClassifier()
#xgb.fit(x_norm_train,y_train.values.ravel())
#xgb_predicted = xgb.predict(x_norm_test)
#print('XGBoost',accuracy_score(y_test, xgb_predicted))
print('XGBoost',cross_val_score(xgb,pca_train_data, df_label.values.ravel(), cv=10).mean())

Naive Bayes 0.841
KNN 0.9079999999999998
Random Forest 0.9039999999999999
Logistic Regression 0.8230000000000001
SVM 0.905
Decision Tree 0.7929999999999999
XGBoost 0.876


# Applying Gaussian Mixture and Grid Search to improve the accurac

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

x_all = np.r_[df_train,df_test]
print('x_all shape :',x_all.shape)

# USING THE GAUSSIAN MIXTURE MODEL 
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components,covariance_type=cv_type)
        gmm.fit(x_all)
        bic.append(gmm.aic(x_all))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
            
best_gmm.fit(x_all)
gmm_train = best_gmm.predict_proba(df_train)
gmm_test = best_gmm.predict_proba(df_test)


#Random Forest Classifier
rfc = RandomForestClassifier(random_state=99)

#USING GRID SEARCH
n_estimators = [10, 50, 100, 200,400]
max_depth = [3, 10, 20, 40]
param_grid = dict(n_estimators=n_estimators,max_depth=max_depth)

grid_search_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv = 10,scoring='accuracy',n_jobs=-1).fit(gmm_train, df_label.values.ravel())
rfc_best = grid_search_rfc.best_estimator_
print('Random Forest Best Score',grid_search_rfc.best_score_)
print('Random Forest Best Parmas',grid_search_rfc.best_params_)
print('Random Forest Accuracy',cross_val_score(rfc_best,gmm_train, df_label.values.ravel(), cv=10).mean())

#KNN 
knn = KNeighborsClassifier()

#USING GRID SEARCH
n_neighbors=[3,5,6,7,8,9,10]
param_grid = dict(n_neighbors=n_neighbors)

grid_search_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv = 10, n_jobs=-1,scoring='accuracy').fit(gmm_train,df_label.values.ravel())
knn_best = grid_search_knn.best_estimator_
print('KNN Best Score', grid_search_knn.best_score_)
print('KNN Best Params',grid_search_knn.best_params_)
print('KNN Accuracy',cross_val_score(knn_best,gmm_train, df_label.values.ravel(), cv=10).mean())

#SVM
svc = SVC()

#USING GRID SEARCH
parameters = [{'kernel':['linear'],'C':[1,10,100]},
              {'kernel':['rbf'],'C':[1,10,100],'gamma':[0.05,0.0001,0.01,0.001]}]
grid_search_svm = GridSearchCV(estimator=svc, param_grid=parameters, cv = 10, n_jobs=-1,scoring='accuracy').fit(gmm_train, df_label.values.ravel())
svm_best = grid_search_svm.best_estimator_
print('SVM Best Score',grid_search_svm.best_score_)
print('SVM Best Params',grid_search_svm.best_params_)
print('SVM Accuracy',cross_val_score(svm_best,gmm_train, df_label.values.ravel(), cv=10).mean())

x_all shape : (10000, 40)
Random Forest Best Score 0.996
Random Forest Best Parmas {'max_depth': 3, 'n_estimators': 10}
Random Forest Accuracy 0.9960000000000001
KNN Best Score 0.996
KNN Best Params {'n_neighbors': 3}
KNN Accuracy 0.9960000000000001
SVM Best Score 0.996
SVM Best Params {'C': 1, 'kernel': 'linear'}
SVM Accuracy 0.9960000000000001


In [ ]:
rfc_best.fit(gmm_train,df_label.values.ravel())
pred  = rfc_best.predict(gmm_test)
rfc_best_pred = pd.DataFrame(pred)

rfc_best_pred.index += 1

rfc_best_pred.columns = ['Solution']
rfc_best_pred['Id'] = np.arange(1,rfc_best_pred.shape[0]+1)
rfc_best_pred = rfc_best_pred[['Id', 'Solution']]

rfc_best_pred.to_csv('Submission_GMM_RFC.csv',index=False)